In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



data = pd.read_csv('/content/drive/My Drive/ODSL-Limpieza/penal_df.csv')
df = data.copy()
df

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,org_idradactual,exp_cuij,org_idintpri,org_cod_pri,org_descorta_pri,org_descr_Pri,org_idintseg,est_id,est_descr,exp_anio,exp_cuerpos,exp_fojas,exp_detenidos,exp_id,exp_sufijo,exp_fechecho,exp_fecinidem,exp_fecini,exp_fecreg,exp_fecsor,exp_fecelev,exp_letra,oju_id,oju_descr,exp_ultmov,exp_ultmovf,den_id,den_cod_casoMPF,Tipo_ingreso_MPF,org_cod,org_descr,org_descorta,org_den,org_descorta_Den,org_descr_Den,den_flagrante,Comuna,den_barrio,den_calle_id,den_cod_postal,den_fec_hecho,den_fec_ingreso_MPF,den_fec_ini_comi,den_hora_hecho,den_latitud,den_longitud,den_tipo_den,den_loc_id,den_sin_lugar_hecho,den_v_domestica,den_v_genero,den_zon_id,den_zona,filter_.,org_destino_act,PrimarioDen_id,PrimarioDen_id_oju,PrimarioExp_id,PrimarioExp_oju
0,72,01-00042685-3,72,JUZP01,JUZGADO N°1,JUZGADO DE 1RA INSTANCIA EN LO PENAL CONTRAVEN...,unknown,1,ASIGNADO,2016,1,1,0,1928087,0,17/2/2014 00:00,26/7/2016 13:45,25/3/2014 00:00,26/7/2016 13:45,26/7/2016 13:45,NaN,J,198,128 1 PARR - DELITOS ATINENTES A LA PORNOGRAFI...,3373851,3373851,400639,NaN,NaN,MPF,MINISTERIO PUBLICO FISCAL,MPF,unknown,NaN,NaN,NaN,unknown,unknown,unknown,unknown,NaN,NaN,NaN,NaN,unknown,unknown,unknown,unknown,unknown,NaN,NaN,unknown,unknown,1,unknown,1,1,1,1
1,72,01-00039012-3,72,JUZP01,JUZGADO N°1,JUZGADO DE 1RA INSTANCIA EN LO PENAL CONTRAVEN...,unknown,14,ARCHIVADO,2016,1,1,0,1923937,0,NaN,31/10/2014 00:00,19/5/2016 00:00,19/5/2016 15:40,19/5/2016 00:00,NaN,J,211,149 BIS - AMENAZAS - CP (P/ L 2303),unknown,unknown,440161,NaN,NaN,MPF,MINISTERIO PUBLICO FISCAL,MPF,unknown,NaN,NaN,NaN,unknown,unknown,unknown,unknown,NaN,NaN,NaN,NaN,unknown,unknown,unknown,unknown,unknown,NaN,NaN,unknown,unknown,1,unknown,1,1,1,1
2,72,01-00035028-8,72,JUZP01,JUZGADO N°1,JUZGADO DE 1RA INSTANCIA EN LO PENAL CONTRAVEN...,unknown,1,ASIGNADO,2016,1,1,0,1919296,0,3/12/2014 00:00,3/3/2016 09:40,23/2/2015 00:00,3/3/2016 09:40,3/3/2016 09:40,NaN,J,211,149 BIS - AMENAZAS - CP (P/ L 2303),3035939,3035939,461090,NaN,NaN,MPF,MINISTERIO PUBLICO FISCAL,MPF,unknown,NaN,NaN,NaN,unknown,unknown,unknown,unknown,NaN,NaN,NaN,NaN,unknown,unknown,unknown,unknown,unknown,NaN,NaN,unknown,unknown,1,unknown,1,1,1,1
3,72,01-00036824-1,72,JUZP01,JUZGADO N°1,JUZGADO DE 1RA INSTANCIA EN LO PENAL CONTRAVEN...,unknown,14,ARCHIVADO,2016,1,1,0,1921403,0,13/4/2015 00:00,14/5/2015 00:00,8/4/2016 00:00,8/4/2016 12:20,8/4/2016 00:00,NaN,J,198,128 1 PARR - DELITOS ATINENTES A LA PORNOGRAFI...,14472595,14472595,477721,NaN,NaN,MPF,MINISTERIO PUBLICO FISCAL,MPF,unknown,NaN,NaN,NaN,unknown,unknown,unknown,unknown,NaN,NaN,NaN,NaN,unknown,unknown,unknown,unknown,unknown,NaN,NaN,unknown,unknown,1,unknown,1,1,1,1
4,72,01-00034124-6,72,JUZP01,JUZGADO N°1,JUZGADO DE 1RA INSTANCIA EN LO PENAL CONTRAVEN...,unknown,14,ARCHIVADO,2016,1,1,0,1918228,0,14/5/2015 00:00,19/5/2015 00:00,29/2/2016 00:00,15/2/2016 10:25,15/2/2016 00:00,NaN,J,217,183 - DAÑOS,3020731,3020731,478747,NaN,NaN,MPF,MINISTERIO PUBLICO FISCAL,MPF,unknown,NaN,NaN,NaN,unknown,unknown,unknown,unknown,NaN,NaN,NaN,NaN,unknown,unknown,unknown,unknown,unknown,NaN,NaN,unknown,unknown,1,unknown,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37131,1001,01-00051596-1,1001,SDPCYF,S/D,S/D,unknown,14,ARCHIVADO,2016,1,1,0,1938211,0,23/4/2016 00:00,25/11/2016 10:05,24/11/2016 00:00,25/11/2016 10:05,25/11/2016 10:05,NaN,J,198,128 1 PARR - DELITOS ATINENTES A LA PORNOGRAFI...,14507930,14507930,583730,MPF00128643,DENUNCIA,MPF,MINISTERIO PUBLICO FISCAL,MPF,0,MPF,MINISTERIO PUBLICO FISCAL,NO,unknown,unknown,unknown,unknown,23/4/2016 00:00,24/11/2016 00:00,NaN,23/4/2016 00:00,unknown,unknown,2,248,0,NO,NaN,unknown,unknown,1,unknown,1,1,1,1
37132,1001,01-00046682-0,1001,SDPCYF,S/D,S/D,unknown,1,ASIGNADO,2016,1,1,0,1932649,0,18/6/2016 00:00,22/9/2016 08:35,9/9/2016 00:00,22/9/2016 08:35,22/9/2016 08:35,NaN,

In [3]:
df.org_idradactual.unique()

array([  72,   73,   74,   75,   76,   77,   78,   79,   80,   81,   82,
         83,   84,   85,   86,   87,   88,   89,   90,   91,   92,   93,
         94,   95,   96,   97,   98,   99,  100,  101,  102, 1001])

In [4]:
df.org_idintpri.unique()

array([  72,   97,   86,   94,   74,   77,   84,   93,   83,   95,   87,
         82,   76,   78,   75,   79,   81,  102,   99,   91,   88,   96,
        101,   98,   73,   80,   89,  100,   85,   92,   90, 1001])

In [5]:
df.org_descorta_pri.value_counts()

JUZGADO N°3     1730
JUZGADO N°11    1724
JUZGADO N°4     1306
JUZGADO N°24    1291
JUZGADO N°15    1290
JUZGADO N°23    1285
JUZGADO N°9     1269
JUZGADO N°28    1266
JUZGADO N°6     1262
JUZGADO N°30    1230
JUZGADO N°29    1211
JUZGADO N°27    1211
JUZGADO N°25    1210
JUZGADO N°12    1187
JUZGADO N°22    1177
JUZGADO N°19    1173
JUZGADO N°18    1153
JUZGADO N°14    1144
JUZGADO N°20    1126
JUZGADO N°31    1124
JUZGADO N°13    1122
JUZGADO N°7     1121
JUZGADO N°21    1113
JUZGADO N°8     1112
JUZGADO N°16    1102
JUZGADO N°1     1094
JUZGADO N°26    1055
JUZGADO N°17    1025
JUZGADO N°5     1011
JUZGADO N°10     973
JUZGADO N°2      963
S/D               76
Name: org_descorta_pri, dtype: int64

**Limpiamos todos los valores no númericos de la columna org_descorta_pri que hace referencia al juzgado al que se le asignó el expediente cuando se judicializó**


In [0]:
def clean_org_descorta_pri(df):

    df['org_descorta_pri'] = df['org_descorta_pri'].replace('S/D', np.nan)

    df.dropna(subset =['org_descorta_pri'], inplace = True)

    df.reset_index(drop = True ,inplace = True)
    
    df['org_descorta_pri'] = df['org_descorta_pri'].str.extract('(\d+)', expand=False).astype(int) 
    
    return df

df = clean_org_descorta_pri(df)

**Reemplazamos los códigos en las columnas org_idintpri y org_idradactual que hacen referencia al número del Juzgado**


In [0]:
df_juzgados = df[['org_idintpri', 'org_descorta_pri']].drop_duplicates()
d = {}
for i in df_juzgados['org_idintpri']:
    d[i] = [df_juzgados['org_descorta_pri'][j] for j in df_juzgados[df_juzgados['org_idintpri']==i].index]

In [8]:
d

{72: [1],
 73: [2],
 74: [3],
 75: [4],
 76: [5],
 77: [6],
 78: [7],
 79: [8],
 80: [9],
 81: [10],
 82: [11],
 83: [12],
 84: [13],
 85: [14],
 86: [15],
 87: [16],
 88: [17],
 89: [18],
 90: [19],
 91: [20],
 92: [21],
 93: [22],
 94: [23],
 95: [24],
 96: [25],
 97: [26],
 98: [27],
 99: [28],
 100: [29],
 101: [30],
 102: [31]}

In [0]:
df["org_idintpri"].replace(d, inplace=True)
df["org_idradactual"].replace(d, inplace=True)

In [10]:
df.head()

,org_idradactual,exp_cuij,org_idintpri,org_cod_pri,org_descorta_pri,org_descr_Pri,org_idintseg,est_id,est_descr,exp_anio,exp_cuerpos,exp_fojas,exp_detenidos,exp_id,exp_sufijo,exp_fechecho,exp_fecinidem,exp_fecini,exp_fecreg,exp_fecsor,exp_fecelev,exp_letra,oju_id,oju_descr,exp_ultmov,exp_ultmovf,den_id,den_cod_casoMPF,Tipo_ingreso_MPF,org_cod,org_descr,org_descorta,org_den,org_descorta_Den,org_descr_Den,den_flagrante,Comuna,den_barrio,den_calle_id,den_cod_postal,den_fec_hecho,den_fec_ingreso_MPF,den_fec_ini_comi,den_hora_hecho,den_latitud,den_longitud,den_tipo_den,den_loc_id,den_sin_lugar_hecho,den_v_domestica,den_v_genero,den_zon_id,den_zona,filter_.,org_destino_act,PrimarioDen_id,PrimarioDen_id_oju,PrimarioExp_id,PrimarioExp_oju
0,1,01-00042685-3,1,JUZP01,1,JUZGADO DE 1RA INSTANCIA EN LO PENAL CONTRAVEN...,unknown,1,ASIGNADO,2016,1,1,0,1928087,0,17/2/2014 00:00,26/7/2016 13:45,25/3/2014 00:00,26/7/2016 13:45,26/7/2016 13:45,NaN,J,198,128 1 PARR - DELITOS ATINENTES A LA PORNOGRAFI...,3373851,3373851,400639,NaN,NaN,MPF,MINISTERIO PUBLICO FISCAL,MPF,unknown,NaN,NaN,NaN,unknown,unknown,unknown,unknown,NaN,NaN,NaN,NaN,unknown,unknown,unknown,unknown,unknown,NaN,NaN,unknown,unknown,1,unknown,1,1,1,1
1,1,01-00039012-3,1,JUZP01,1,JUZGADO DE 1RA INSTANCIA EN LO PENAL CONTRAVEN...,unknown,14,ARCHIVADO,2016,1,1,0,1923937,0,NaN,31/10/2014 00:00,19/5/2016 00:00,19/5/2016 15:40,19/5/2016 00:00,NaN,J,211,149 BIS - AMENAZAS - CP (P/ L 2303),unknown,unknown,440161,NaN,NaN,MPF,MINISTERIO PUBLICO FISCAL,MPF,unknown,NaN,NaN,NaN,unknown,unknown,unknown,unknown,NaN,NaN,NaN,NaN,unknown,unknown,unknown,unknown,unknown,NaN,NaN,unknown,unknown,1,unknown,1,1,1,1
2,1,01-00035028-8,1,JUZP01,1,JUZGADO DE 1RA INSTANCIA EN LO PENAL CONTRAVEN...,unknown,1,ASIGNADO,2016,1,1,0,1919296,0,3/12/2014 00:00,3/3/2016 09:40,23/2/2015 00:00,3/3/2016 09:40,3/3/2016 09:40,NaN,J,211,149 BIS - AMENAZAS - CP (P/ L 2303),3035939,3035939,461090,NaN,NaN,MPF,MINISTERIO PUBLICO FISCAL,MPF,unknown,NaN,NaN,NaN,unknown,unknown,unknown,unknown,NaN,NaN,NaN,NaN,unknown,unknown,unknown,unknown,unknown,NaN,NaN,unknown,unknown,1,unknown,1,1,1,1
3,1,01-00036824-1,1,JUZP01,1,JUZGADO DE 1RA INSTANCIA EN LO PENAL CONTRAVEN...,unknown,14,ARCHIVADO,2016,1,1,0,1921403,0,13/4/2015 00:00,14/5/2015 00:00,8/4/2016 00:00,8/4/2016 12:20,8/4/2016 00:00,NaN,J,198,128 1 PARR - DELITOS ATINENTES A LA PORNOGRAFI...,14472595,14472595,477721,NaN,NaN,MPF,MINISTERIO PUBLICO FISCAL,MPF,unknown,NaN,NaN,NaN,unknown,unknown,unknown,unknown,NaN,NaN,NaN,NaN,unknown,unknown,unknown,unknown,unknown,NaN,NaN,unknown,unknown,1,unknown,1,1,1,1
4,1,01-00034124-6,1,JUZP01,1,JUZGADO DE 1RA INSTANCIA EN LO PENAL CONTRAVEN...,unknown,14,ARCHIVADO,2016,1,1,0,1918228,0,14/5/2015 00:00,19/5/2015 00:00,29/2/2016 00:00,15/2/2016 10:25,15/2/2016 00:00,NaN,J,217,183 - DAÑOS,3020731,3020731,478747,NaN,NaN,MPF,MINISTERIO PUBLICO FISCAL,MPF,unknown,NaN,NaN,NaN,unknown,unknown,unknown,unknown,NaN,NaN,NaN,NaN,unknown,unknown,unknown,unknown,unknown,NaN,NaN,unknown,unknown,1,unknown,1,1,1,1


**De la columna descripción delitos, extraemos la categoría del delito; el artículo del Código Penal al que hace referencia y la pena estimada según el Código Penal**


In [0]:
categoria_delitos = []
articulo_codigo_penal = [] #puede ser código penal o alguna ley nacional.
pena_estimada = [] #promedio de meses

for i in df.oju_descr:
    
    if '149 BIS' in i:
        categoria_delitos.append('Amenazas')
        articulo_codigo_penal.append('149 BIS')
        pena_estimada.append(15)
    
    elif '183' in i:
        categoria_delitos.append('Daños')
        articulo_codigo_penal.append('183')
        pena_estimada.append(6)
    
    elif '128 1' in i:
        categoria_delitos.append('Pornografia infantil')
        articulo_codigo_penal.append('128')
        pena_estimada.append(54)
    
    elif '89' in i:
        categoria_delitos.append('Lesiones leves')
        articulo_codigo_penal.append('89')
        pena_estimada.append(6)
    
    elif '239' in i:
        categoria_delitos.append('Resistencia a la autoridad')
        articulo_codigo_penal.append('239')
        pena_estimada.append(6)
    
    elif '5 C' in i:
        categoria_delitos.append('Comercio de estupefacientes')
        articulo_codigo_penal.append('5 C')
        pena_estimada.append(21)
    
    elif '14 1' in i or '14 2' in i:
        categoria_delitos.append('Tenencia de estupefacientes')
        articulo_codigo_penal.append('14 1')
        pena_estimada.append(12)
    
    elif '92' in i:
        categoria_delitos.append('Lesiones graves')
        articulo_codigo_penal.append('92')
        pena_estimada.append(24)
    
    elif '292 1' in i or '296' in i:
        categoria_delitos.append('Falsificacion DNI')
        articulo_codigo_penal.append('292')
        pena_estimada.append(42)  
        
    elif '292 2°PARR' in i:
        categoria_delitos.append('Falsificacion documento privado')
        articulo_codigo_penal.append('292')
        pena_estimada.append(15)  
        
    elif 'INCUMPLIMIENTO DE LOS DEBERES DE ASISTENCIA FAMILIAR' in i or '1 - LN 13.944 ' in i:
        categoria_delitos.append('Incumplimiento asistencia familiar')
        articulo_codigo_penal.append('13.944')
        pena_estimada.append(12) 
        
    elif '181 INC. 1'in i:
        categoria_delitos.append('Usurpacion')
        articulo_codigo_penal.append('181')
        pena_estimada.append(22)   
    
    elif 'PRESUNTA COMISION DELITO' in i:
        categoria_delitos.append('Presunta comision delito')
        articulo_codigo_penal.append('nan')
        pena_estimada.append(0)   
    
    elif '301BIS' in i:
        categoria_delitos.append('Apuestas ilegales')
        articulo_codigo_penal.append('301 BIS')
        pena_estimada.append(44)   
    
    elif '94  -' in i:
        categoria_delitos.append('Lesiones culposas')
        articulo_codigo_penal.append('94')
        pena_estimada.append(18)          
    
    elif '1 1er párr' in i:
        categoria_delitos.append('Impedimento de contacto de menor con padre no conviviente')
        articulo_codigo_penal.append('2470')
        pena_estimada.append(6)    
    
    elif '237  - ' in i:
        categoria_delitos.append('Atentado contra la autoridad')
        articulo_codigo_penal.append('237')
        pena_estimada.append(6)
        
    elif '150' in i:
        categoria_delitos.append('Violacion de domicilio')
        articulo_codigo_penal.append('150')
        pena_estimada.append(15)
    
    elif '189 BIS' in i:
        categoria_delitos.append('Posesion de arma de fuego')
        articulo_codigo_penal.append('189 BIS (2)')
        pena_estimada.append(15)
    
    elif '131' in i:
        categoria_delitos.append('Grooming')
        articulo_codigo_penal.append('131')
        pena_estimada.append(27)
    
    else:
        categoria_delitos.append('otros')
        articulo_codigo_penal.append('otros')
        pena_estimada.append(0)

In [0]:
def join_categoria(df):
    
    df_cat_delitos = pd.DataFrame(np.column_stack([categoria_delitos, articulo_codigo_penal, pena_estimada]), 
                                  columns = ['descripcion_delitos', 'articulo_codigo_penal','pena_estimada'])
    df_nuevo = df.join(df_cat_delitos, how = 'right')
    
    return df_nuevo

df = join_categoria(df)

In [13]:
def null_values(df):
    nulls = pd.DataFrame((df.isnull().sum().sort_values()/len(df)*100).round(2),columns=['pct_null'])
    nulls = nulls.loc[nulls['pct_null']>0]
    nulls = nulls.sort_values(by = 'pct_null', ascending = False)
    return nulls

nulls = null_values(df)
nulls

,pct_null
exp_fecelev,97.20
den_v_genero,85.03
den_hora_hecho,80.21
den_cod_casoMPF,69.55
exp_fechecho,64.66
den_fec_ini_comi,58.11
Tipo_ingreso_MPF,53.46
exp_fecinidem,37.16
den_fec_hecho,25.63
den_fec_ingreso_MPF,9.75


In [0]:
df.drop(columns = ['org_cod_pri','exp_fecelev','den_v_genero','den_hora_hecho','den_cod_casoMPF','den_fec_ini_comi', 'exp_fechecho','den_fec_ini_comi','Tipo_ingreso_MPF'], inplace = True)

**Se eliminaron las siguientes columnas:**



*   org_cod_pri : Código alfanumérico del nombre del juzgado al que se le asignó el expediente ya que es información que ya está en otras columnas.
*   exp_facelev : Fecha de elevación a Cámara en los casos de apelación
*   den_v_genero: Indica si el caso involucra cuestiones de violencia de género
*   den_hora_hecho : Hora del hecho
*   den_cod_casoMPF : Denota acumulación de denuncias en un caso (MPF)
*   den_fec_ini_comi : Fecha  ingreso en Comisaría
*   exp_fechecho : Fecha de ocurrencia del hecho investigado
*   den_fec_ini_comi : Fecha  ingreso en Comisaría
*   Tipo_ingreso_MPF : Forma de ingreso del hecho













In [0]:
df = df.rename(columns = {'den_fec_hecho' : 'fecha_denuncia_hecho','exp_fecini': 'fecha_inicio_investigacion','exp_fecinidem' : 'fecha_inicio_demanda', 
                          'exp_fecsor':'fecha_sorteo_expediente','exp_fecreg':'fecha_registro_juzgado'})


In [16]:
def convert_to_datetime(df):

  df['fecha_denuncia_hecho'] = pd.to_datetime(df.fecha_denuncia_hecho, infer_datetime_format=True)
  df['fecha_inicio_investigacion'] = pd.to_datetime(df.fecha_inicio_investigacion, infer_datetime_format=True)
  df['fecha_inicio_demanda'] = pd.to_datetime(df.fecha_inicio_demanda, infer_datetime_format=True)
  df['fecha_sorteo_expediente'] = pd.to_datetime(df.fecha_sorteo_expediente, infer_datetime_format=True)
  df['fecha_registro_juzgado'] = pd.to_datetime(df.fecha_registro_juzgado, infer_datetime_format=True)
  return df

df = convert_to_datetime(df)
df.head()

,org_idradactual,exp_cuij,org_idintpri,org_descorta_pri,org_descr_Pri,org_idintseg,est_id,est_descr,exp_anio,exp_cuerpos,exp_fojas,exp_detenidos,exp_id,exp_sufijo,fecha_inicio_demanda,fecha_inicio_investigacion,fecha_registro_juzgado,fecha_sorteo_expediente,exp_letra,oju_id,oju_descr,exp_ultmov,exp_ultmovf,den_id,org_cod,org_descr,org_descorta,org_den,org_descorta_Den,org_descr_Den,den_flagrante,Comuna,den_barrio,den_calle_id,den_cod_postal,fecha_denuncia_hecho,den_fec_ingreso_MPF,den_latitud,den_longitud,den_tipo_den,den_loc_id,den_sin_lugar_hecho,den_v_domestica,den_zon_id,den_zona,filter_.,org_destino_act,PrimarioDen_id,PrimarioDen_id_oju,PrimarioExp_id,PrimarioExp_oju,descripcion_delitos,articulo_codigo_penal,pena_estimada
0,1,01-00042685-3,1,1,JUZGADO DE 1RA INSTANCIA EN LO PENAL CONTRAVEN...,unknown,1,ASIGNADO,2016,1,1,0,1928087,0,2016-07-26 13:45:00,2014-03-25,2016-07-26 13:45:00,2016-07-26 13:45:00,J,198,128 1 PARR - DELITOS ATINENTES A LA PORNOGRAFI...,3373851,3373851,400639,MPF,MINISTERIO PUBLICO FISCAL,MPF,unknown,NaN,NaN,NaN,unknown,unknown,unknown,unknown,NaT,NaN,unknown,unknown,unknown,unknown,unknown,NaN,unknown,unknown,1,unknown,1,1,1,1,Pornografia infantil,128,54
1,1,01-00039012-3,1,1,JUZGADO DE 1RA INSTANCIA EN LO PENAL CONTRAVEN...,unknown,14,ARCHIVADO,2016,1,1,0,1923937,0,2014-10-31 00:00:00,2016-05-19,2016-05-19 15:40:00,2016-05-19 00:00:00,J,211,149 BIS - AMENAZAS - CP (P/ L 2303),unknown,unknown,440161,MPF,MINISTERIO PUBLICO FISCAL,MPF,unknown,NaN,NaN,NaN,unknown,unknown,unknown,unknown,NaT,NaN,unknown,unknown,unknown,unknown,unknown,NaN,unknown,unknown,1,unknown,1,1,1,1,Amenazas,149 BIS,15
2,1,01-00035028-8,1,1,JUZGADO DE 1RA INSTANCIA EN LO PENAL CONTRAVEN...,unknown,1,ASIGNADO,2016,1,1,0,1919296,0,2016-03-03 09:40:00,2015-02-23,2016-03-03 09:40:00,2016-03-03 09:40:00,J,211,149 BIS - AMENAZAS - CP (P/ L 2303),3035939,3035939,461090,MPF,MINISTERIO PUBLICO FISCAL,MPF,unknown,NaN,NaN,NaN,unknown,unknown,unknown,unknown,NaT,NaN,unknown,unknown,unknown,unknown,unknown,NaN,unknown,unknown,1,unknown,1,1,1,1,Amenazas,149 BIS,15
3,1,01-00036824-1,1,1,JUZGADO DE 1RA INSTANCIA EN LO PENAL CONTRAVEN...,unknown,14,ARCHIVADO,2016,1,1,0,1921403,0,2015-05-14 00:00:00,2016-04-08,2016-04-08 12:20:00,2016-04-08 00:00:00,J,198,128 1 PARR - DELITOS ATINENTES A LA PORNOGRAFI...,14472595,14472595,477721,MPF,MINISTERIO PUBLICO FISCAL,MPF,unknown,NaN,NaN,NaN,unknown,unknown,unknown,unknown,NaT,NaN,unknown,unknown,unknown,unknown,unknown,NaN,unknown,unknown,1,unknown,1,1,1,1,Pornografia infantil,128,54
4,1,01-00034124-6,1,1,JUZGADO DE 1RA INSTANCIA EN LO PENAL CONTRAVEN...,unknown,14,ARCHIVADO,2016,1,1,0,1918228,0,2015-05-19 00:00:00,2016-02-29,2016-02-15 10:25:00,2016-02-15 00:00:00,J,217,183 - DAÑOS,3020731,3020731,478747,MPF,MINISTERIO PUBLICO FISCAL,MPF,unknown,NaN,NaN,NaN,unknown,unknown,unknown,unknown,NaT,NaN,unknown,unknown,unknown,unknown,unknown,NaN,unknown,unknown,1,unknown,1,1,1,1,Daños,183,6


In [19]:
df_fechas = df[['exp_anio','fecha_inicio_demanda',	'fecha_inicio_investigacion',	'fecha_registro_juzgado',	'fecha_sorteo_expediente']]
df_fechas

,exp_anio,fecha_inicio_demanda,fecha_inicio_investigacion,fecha_registro_juzgado,fecha_sorteo_expediente
0,2016,2016-07-26 13:45:00,2014-03-25,2016-07-26 13:45:00,2016-07-26 13:45:00
1,2016,2014-10-31 00:00:00,2016-05-19,2016-05-19 15:40:00,2016-05-19 00:00:00
2,2016,2016-03-03 09:40:00,2015-02-23,2016-03-03 09:40:00,2016-03-03 09:40:00
3,2016,2015-05-14 00:00:00,2016-04-08,2016-04-08 12:20:00,2016-04-08 00:00:00
4,2016,2015-05-19 00:00:00,2016-02-29,2016-02-15 10:25:00,2016-02-15 00:00:00
...,...,...,...,...,...
37055,2017,2017-10-26 11:00:00,2017-09-20,2017-10-26 12:23:00,2017-10-26 11:00:00
37056,2019,NaT,NaT,2019-09-13 13:23:00,2019-09-30 11:58:00
37057,2019,NaT,NaT,2019-09-12 14:27:00,2019-09-18 15:31:00
37058,2019,NaT,NaT,2019-10-01 13:05:00,2019-10-01 13:05:00


In [21]:
df_fechas.isnull().sum()

exp_anio                          0
fecha_inicio_demanda          13772
fecha_inicio_investigacion     1469
fecha_registro_juzgado            0
fecha_sorteo_expediente           0
dtype: int64

In [23]:
df_fechas[df_fechas.exp_anio == 2019].isnull().sum()

exp_anio                          0
fecha_inicio_demanda          10257
fecha_inicio_investigacion     1132
fecha_registro_juzgado            0
fecha_sorteo_expediente           0
dtype: int64